# Run Agent
- Implement a workflow to write a daily AI newsletter
- see README.md for details


In [1]:
import os
import yaml
import dotenv
import logging
import json
import yaml
from datetime import datetime
import time
import random
import glob
import pickle
import sqlite3

from pathlib import Path

import asyncio
import nest_asyncio

import pydantic
from pydantic import BaseModel, Field, RootModel
from typing import Dict, TypedDict, Type, List, Optional, Any, Iterable, Text
from dataclasses import dataclass, field
from enum import Enum

import numpy as np
import pandas as pd

import pandas as pd
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
import hdbscan

import openai
from openai import AsyncOpenAI

import agents
from agents.exceptions import InputGuardrailTripwireTriggered
from agents import (Agent, Runner, Tool, OpenAIResponsesModel, 
                    ModelSettings, FunctionTool, InputGuardrail, GuardrailFunctionOutput,
                    SQLiteSession, set_default_openai_api, set_default_openai_client
                   )


import tenacity
from tenacity import retry, stop_after_attempt, wait_exponential, retry_if_exception_type

from IPython.display import HTML, Image, Markdown, display

from log_handler import SQLiteLogHandler, setup_sqlite_logging, sanitize_error_for_logging
from config import LOGDB
from llm import LLMagent, LangfuseClient  # methods to apply prompts async to large batches
from db import Url 

from fetch import Fetcher # fetch news urls
from newsletter_state import NewsletterAgentState, StepStatus
from news_agent import NewsletterAgent


In [2]:
print(f"OpenAI:            {openai.__version__}")
print(f"OpenAI Agents SDK  {agents.__version__}")
print(f"Pydantic           {pydantic.__version__}")


OpenAI:            1.109.0
OpenAI Agents SDK  0.3.1
Pydantic           2.11.9


In [3]:
dotenv.load_dotenv()

# to run async in jupyter notebook
nest_asyncio.apply()

# verbose OpenAI console logging if something doesn't work
# logging.basicConfig(level=logging.DEBUG)
# openai_logger = logging.getLogger("openai")
# openai_logger.setLevel(logging.DEBUG)


In [4]:
# modules create a default logger, or we can pass this logger

def setup_logging(session_id: str = "default", db_path: str = "agent_logs.db") -> logging.Logger:
    """Set up logging to console and SQLite database."""

    # Create logger
    logging.basicConfig(level=logging.INFO)

    logger = logging.getLogger(f"NewsletterAgent.{session_id}")
    logger.setLevel(logging.INFO)

    # Clear any existing handlers
    logger.handlers.clear()

    # Console handler
    console_handler = logging.StreamHandler()
    console_handler.setLevel(logging.INFO)
    console_formatter = logging.Formatter(
        '%(asctime)s | %(name)s | %(levelname)s | %(message)s',
        datefmt='%H:%M:%S'
    )
    console_handler.setFormatter(console_formatter)

    # SQLite handler
    sqlite_handler = SQLiteLogHandler(db_path)
    sqlite_handler.setLevel(logging.INFO)
    sqlite_formatter = logging.Formatter('%(message)s')
    sqlite_handler.setFormatter(sqlite_formatter)

    # Add handlers to logger
    logger.addHandler(console_handler)
    logger.addHandler(sqlite_handler)

    # Prevent propagation to root logger
    logger.propagate = False

    return logger

logger = setup_logging("newsletter_agent", "test_logs.db")

# Log some test messages
logger.info("Test info message", extra={
    'step_name': 'test_step',
    'agent_session': 'demo_session'
})

logger.warning("Test warning message", extra={
    'step_name': 'test_step',
    'agent_session': 'demo_session'
})

logger.error("Test error message", extra={
    'step_name': 'error_step',
    'agent_session': 'demo_session'
})

sanitize_error_for_logging("log with some bad stuff for the filter: sk-proj-123456789012345678901234567890123456789012345678")

18:00:21 | NewsletterAgent.newsletter_agent | INFO | Test info message
18:00:21 | NewsletterAgent.newsletter_agent | WARNING | Test warning message
18:00:21 | NewsletterAgent.newsletter_agent | ERROR | Test error message


'log with some bad stuff for the filter: [API_KEY_REDACTED]'

# Run Agent Worfklow

In [5]:
print("🚀 Creating NewsletterAgent...")

do_download=True
process_since=None
# process_since='2025-10-05 18:30:00'

try:
    # set up state
    session_id = 'test_newsletter_20251009073511683688'
    step_name = 'step_07_select_sections'
    del session_id, step_name
except Exception as e:
    print(e)

api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("OPENAI_API_KEY environment variable not set")

# Set up OpenAI client for the agents SDK
set_default_openai_client(AsyncOpenAI(api_key=api_key))

# Create agent with persistent state
if 'session_id' in vars():
    # load state from db for session_id and state
    print("session_id is defined")
    print(session_id)
    state = NewsletterAgentState(session_id=session_id, 
                                 db_path="newsletter_agent.db", 
                                 do_download=do_download,
                                 process_since=process_since,
                                 verbose=False
                                )
    state = state.load_from_db(step_name)
    agent = NewsletterAgent(session_id=session_id, state=state, verbose=True, timeout=30)    
else:
    # create new session
    print("session_id is not defined")
    timestamp = datetime.now().strftime("%Y%m%d%H%M%S%f")    
    session_id = f"test_newsletter_{timestamp}"
    print(session_id)
    state = NewsletterAgentState(session_id=session_id, 
                                 db_path="newsletter_agent.db",
                                 do_download=do_download,
                                 process_since=process_since,
                                 verbose=False
                                ) 
    agent = NewsletterAgent(session_id=session_id, state=state, verbose=False, timeout=30)
    state.serialize_to_db("initialize")

🚀 Creating NewsletterAgent...
session_id is not defined
test_newsletter_20251009180028201669


In [6]:
agent.state.get_status()


{'headlines': {'total': 0},
 'sources': {'config_file': 'sources.yaml', 'loaded_sources': 0},
 'topics': {'cluster_topics': 0, 'topics': []},
 'workflow': {'current_step': 'step_01_fetch_urls',
  'workflow_complete': False,
  'workflow_status': 'not_started',
  'workflow_status_message': '',
  'progress_percentage': 0.0,
  'max_edits': 2,
  'concurrency': 16,
  'do_download': True,
  'process_since': None},
 'processing': {'topic_clusters': 0,
  'newsletter_sections': 0,
  'final_newsletter_length': 0}}

In [7]:
# User prompt to run workflow
user_prompt = "Show the workflow status"

print(f"\n📝 User prompt: '{user_prompt}'")
print("=" * 80)

# Run the agent with persistent state
start_time = time.time()
result = await agent.run_step(user_prompt)
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


📝 User prompt: 'Show the workflow status'


18:00:36 | NewsletterAgent.test_newsletter_20251009180028201669 | INFO | Starting check_workflow_status
18:00:36 | NewsletterAgent.test_newsletter_20251009180028201669 | INFO | Completed check_workflow_status


⏱️  Total execution time: 7.61s
📊 Final result:
Current workflow status:
- Progress: 0.0% (0/9 complete)
- 0 steps completed, 0 started, 0 failed, 9 not started
- Next step: Step 1 — Fetch Urls

Step-by-step status:
1. Fetch Urls: not_started
2. Filter Urls: not_started
3. Download Articles: not_started
4. Extract Summaries: not_started
5. Rate Articles: not_started
6. Cluster By Topic: not_started
7. Select Sections: not_started
8. Draft Sections: not_started
9. Finalize Newsletter: not_started

Would you like me to start Step 1 (gather URLs) now, or resume from a specific step?


In [8]:
# User prompt to run a workflow step
user_prompt = "Run step 1, fetch urls"

print(f"\n📝 User prompt: '{user_prompt}'")
print("=" * 80)

# Run the agent with persistent state
start_time = time.time()
result = await agent.run_step(user_prompt)
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)



📝 User prompt: 'Run step 1, fetch urls, if any failed state which one and the url'


18:00:43 | NewsletterAgent.test_newsletter_20251009180028201669 | INFO | Starting check_workflow_status
18:00:43 | NewsletterAgent.test_newsletter_20251009180028201669 | INFO | Completed check_workflow_status
18:00:45 | NewsletterAgent.test_newsletter_20251009180028201669 | INFO | Starting Step 1: Gather URLs
18:00:45 | NewsletterAgent.test_newsletter_20251009180028201669 | INFO | Cleaning download/sources: 
18:00:45 | NewsletterAgent.test_newsletter_20251009180028201669 | INFO | Successfully cleaned download/sources
18:00:45 | NewsletterAgent.test_newsletter_20251009180028201669 | INFO | Cleaning download/text: 
18:00:45 | NewsletterAgent.test_newsletter_20251009180028201669 | INFO | Successfully cleaned download/text
18:00:45 | NewsletterAgent.test_newsletter_20251009180028201669 | INFO | Cleaning download/html: 
18:00:45 | NewsletterAgent.test_newsletter_20251009180028201669 | INFO | Successfully cleaned download/html
2025-10-09 18:00:45,592 - fetcher_5787722448 - INFO - [fetcher_in

▶ Starting Step 1: step_01_fetch_urls


2025-10-09 18:00:45,784 - fetcher_5787722448 - INFO - [fetch_rss] RSS fetch successful for New York Times: 24 articles
2025-10-09 18:00:45,785 - fetcher_5787722448 - INFO - [fetch_html] Fetching HTML from Reddit: https://www.reddit.com/r/AI_Agents+ArtificialInteligence+Automate+ChatGPT+ChatGPTCoding+Futurology+MachineLearning+OpenAI+ProgrammerHumor+accelerate+aiArt+aivideo+artificial+deeplearning+learnmachinelearning+programming+singularity+tech+technews+technology/top/?sort=top&t=day
2025-10-09 18:00:45,840 - fetcher_5787722448 - INFO - [fetch_rss] RSS fetch successful for HackerNoon: 50 articles
2025-10-09 18:00:45,841 - fetcher_5787722448 - INFO - [fetch_html] Fetching HTML from Techmeme: https://www.techmeme.com/river
2025-10-09 18:00:45,943 - fetcher_5787722448 - INFO - [fetch_rss] RSS fetch successful for Ars Technica: 20 articles
2025-10-09 18:00:45,944 - fetcher_5787722448 - INFO - [fetch_rss] Fetching RSS from The Register: https://www.theregister.com/software/ai_ml/headlines.

2025-10-09 18:00:48,625 - fetcher_5787722448 - INFO - Starting scrape_source https://www.reddit.com/r/AI_Agents+ArtificialInteligence+Automate+ChatGPT+ChatGPTCoding+Futurology+MachineLearning+OpenAI+ProgrammerHumor+accelerate+aiArt+aivideo+artificial+deeplearning+learnmachinelearning+programming+singularity+tech+technews+technology/top/?sort=top&t=day, Reddit
2025-10-09 18:00:48,625 - fetcher_5787722448 - INFO - scrape_url(https://www.reddit.com/r/AI_Agents+ArtificialInteligence+Automate+ChatGPT+ChatGPTCoding+Futurology+MachineLearning+OpenAI+ProgrammerHumor+accelerate+aiArt+aivideo+artificial+deeplearning+learnmachinelearning+programming+singularity+tech+technews+technology/top/?sort=top&t=day)
2025-10-09 18:00:48,625 - fetcher_5787722448 - INFO - scraping https://www.reddit.com/r/AI_Agents+ArtificialInteligence+Automate+ChatGPT+ChatGPTCoding+Futurology+MachineLearning+OpenAI+ProgrammerHumor+accelerate+aiArt+aivideo+artificial+deeplearning+learnmachinelearning+programming+singularity+

...

2025-10-09 18:01:16,061 - fetcher_5787722448 - DEBUG - Found last updated time from document.lastModified: 10/09/2025 15:01:16
2025-10-09 18:01:16,061 - fetcher_5787722448 - DEBUG - Attempting to parse last_updated: '10/09/2025 15:01:16' (type: <class 'str'>)
2025-10-09 18:01:16,062 - fetcher_5787722448 - DEBUG - Parsed datetime: 2025-10-09 15:01:16
2025-10-09 18:01:16,063 - fetcher_5787722448 - DEBUG - Added UTC timezone: 2025-10-09 15:01:16+00:00
2025-10-09 18:01:16,063 - fetcher_5787722448 - DEBUG - Converted to UTC: 2025-10-09 15:01:16+00:00
2025-10-09 18:01:16,064 - fetcher_5787722448 - DEBUG - Formatted last_updated: 2025-10-09T15:01:16Z
2025-10-09 18:01:16,065 - fetcher_5787722448 - INFO - Saving HTML to download/sources/WSJ.html
2025-10-09 18:01:16,081 - fetcher_5787722448 - INFO - [fetch_html] Parsing HTML file: download/sources/WSJ.html
2025-10-09 18:01:16,107 - fetcher_5787722448 - INFO - [fetch_html] Parsed HTML file: download/sources/WSJ.html
2025-10-09 18:01:16,108 - fetc

,source,url
0,Ars Technica,20
1,Bloomberg,28
2,Business Insider,16
3,FT,50
4,Feedly AI,73
5,Hacker News,28
6,HackerNoon,50
7,New York Times,23
8,NewsAPI,95
9,Reddit,50


,source,title,url,published,rss_summary,id
0,Ars Technica,Bank of England warns AI stock bubble rivals 2...,https://arstechnica.com/ai/2025/10/bank-of-eng...,"Wed, 08 Oct 2025 21:18:30 +0000",Central bank says market concentration hasn't ...,0
1,Ars Technica,Vandals deface ads for AI necklaces that liste...,https://arstechnica.com/tech-policy/2025/10/va...,"Wed, 08 Oct 2025 16:07:29 +0000",Critics attacked subway ads to defend human fr...,1
2,Ars Technica,Insurers balk at paying out huge settlements f...,https://arstechnica.com/ai/2025/10/insurers-ba...,"Wed, 08 Oct 2025 13:31:28 +0000","OpenAI, Anthropic consider using investor fund...",2
3,Ars Technica,Ars Live: Is the AI bubble about to pop? Ed Zi...,https://arstechnica.com/ai/2025/10/ars-live-is...,"Tue, 07 Oct 2025 19:15:26 +0000","Our discussion happens today, October 7, at 3:...",3
4,Ars Technica,Dead celebrities are apparently fair game for ...,https://arstechnica.com/ai/2025/10/dj-bruce-le...,"Tue, 07 Oct 2025 17:08:05 +0000","OpenAI's likeness protections for living ""publ...",4
...,...,...,...,...,...,...
750,NewsAPI,DC’s shutdown hasn't stopped the stock market....,https://roanoke.com/news/nation-world/business...,2025-10-08T21:30:00Z,NaN,750
751,NewsAPI,Insurers Balk At Paying Out Huge Settlements F...,https://slashdot.org/submission/17341256/insur...,2025-10-08T20:06:02Z,NaN,751
752,NewsAPI,Financial institutions sound warning about an ...,https://www.whio.com/news/business/is-there-an...,2025-10-08T20:06:13Z,NaN,752
753,NewsAPI,Financial institutions sound warning about an ...,https://www.boston25news.com/news/business/is-...,2025-10-08T18:09:12Z,NaN,753


18:01:20 | NewsletterAgent.test_newsletter_20251009180028201669 | INFO | Completed Step 1: Gathered 864 articles


⏱️  Total execution time: 54.80s
📊 Final result:
Step 1 (Fetch URLs) has been run.

Summary:
- Status: completed successfully.
- Sources scanned: 17 (RSS feeds)
- Headlines found: 864
- Articles stored in persistent state: 755
- Step-level failures: none reported

Notes:
- The fetch tool did not return any failed URLs. If you want, I can list the stored headlines and URLs, or inspect the persistent state in more detail to surface any individual items that were dropped (e.g., duplicates or parse errors).

Next step: Step 2 — Filter URLs to AI-related content. Shall I proceed?


In [10]:
pd.DataFrame(state.headline_data) 


,source,title,url,published,rss_summary,id
0,Ars Technica,Bank of England warns AI stock bubble rivals 2...,https://arstechnica.com/ai/2025/10/bank-of-eng...,"Wed, 08 Oct 2025 21:18:30 +0000",Central bank says market concentration hasn't ...,0
1,Ars Technica,Vandals deface ads for AI necklaces that liste...,https://arstechnica.com/tech-policy/2025/10/va...,"Wed, 08 Oct 2025 16:07:29 +0000",Critics attacked subway ads to defend human fr...,1
2,Ars Technica,Insurers balk at paying out huge settlements f...,https://arstechnica.com/ai/2025/10/insurers-ba...,"Wed, 08 Oct 2025 13:31:28 +0000","OpenAI, Anthropic consider using investor fund...",2
3,Ars Technica,Ars Live: Is the AI bubble about to pop? Ed Zi...,https://arstechnica.com/ai/2025/10/ars-live-is...,"Tue, 07 Oct 2025 19:15:26 +0000","Our discussion happens today, October 7, at 3:...",3
4,Ars Technica,Dead celebrities are apparently fair game for ...,https://arstechnica.com/ai/2025/10/dj-bruce-le...,"Tue, 07 Oct 2025 17:08:05 +0000","OpenAI's likeness protections for living ""publ...",4
...,...,...,...,...,...,...
750,NewsAPI,DC’s shutdown hasn't stopped the stock market....,https://roanoke.com/news/nation-world/business...,2025-10-08T21:30:00Z,NaN,750
751,NewsAPI,Insurers Balk At Paying Out Huge Settlements F...,https://slashdot.org/submission/17341256/insur...,2025-10-08T20:06:02Z,NaN,751
752,NewsAPI,Financial institutions sound warning about an ...,https://www.whio.com/news/business/is-there-an...,2025-10-08T20:06:13Z,NaN,752
753,NewsAPI,Financial institutions sound warning about an ...,https://www.boston25news.com/news/business/is-...,2025-10-08T18:09:12Z,NaN,753


In [11]:
countdf = pd.DataFrame(state.headline_data) \
    .groupby("source") \
    .count()[["id"]] \
    .reset_index() \
    .rename(columns={'id': 'count'}) \
    .sort_values("count", ascending=False)
countdf 


,source,count
10,Techmeme,180
8,NewsAPI,95
4,Feedly AI,73
12,The Register,50
3,FT,50
6,HackerNoon,50
9,Reddit,50
5,Hacker News,28
1,Bloomberg,28
7,New York Times,23


In [12]:
# Run tool directly without LLM processing an input prompt or results
# user_prompt = "Run step 2, filter urls"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

# Run the agent with persistent state
start_time = time.time()
result = await agent.run_tool_direct("filter_urls")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


18:02:34 | NewsletterAgent.test_newsletter_20251009180028201669 | INFO | Starting Step 2: Filter URLs
18:02:34 | NewsletterAgent.test_newsletter_20251009180028201669 | INFO | 🔍 Filtering 755 headlines...
18:02:34 | NewsletterAgent.test_newsletter_20251009180028201669 | INFO | 🔄 Checking for duplicates (all urls without date restrictions)
18:02:35 | NewsletterAgent.test_newsletter_20251009180028201669 | INFO | 🔍 Filtering 755 articles for dupes.
18:02:35 | NewsletterAgent.test_newsletter_20251009180028201669 | INFO | URL deduplication: 465 duplicates filtered, 290 new URLs remain
18:02:35 | NewsletterAgent.test_newsletter_20251009180028201669 | INFO | 🔍 Filtering 755 headlines for AI relevance using LLM...
INFO:llm:Initialized LangfuseClient


▶ Starting Step 2: step_02_filter_urls
checking https://arstechnica.com/ai/2025/10/bank-of-england-warns-ai-stock-bubble-rivals-2000-dotcom-peak/
found url, no cutoff set - treating as duplicate
checking https://arstechnica.com/tech-policy/2025/10/vandals-deface-ads-for-ai-necklaces-that-listen-to-all-your-conversations/
found url, no cutoff set - treating as duplicate
checking https://arstechnica.com/ai/2025/10/insurers-balk-at-paying-out-huge-settlements-for-claims-against-ai-firms/
found url, no cutoff set - treating as duplicate
checking https://arstechnica.com/ai/2025/10/ars-live-is-the-ai-bubble-about-to-pop-a-live-chat-with-ed-zitron/
found url, no cutoff set - treating as duplicate
checking https://arstechnica.com/ai/2025/10/dj-bruce-lee-and-jackass-mr-rogers-dead-celebrities-become-puppets-in-sora-2-videos/
found url, no cutoff set - treating as duplicate
checking https://arstechnica.com/ai/2025/10/openai-wants-to-make-chatgpt-into-a-universal-app-frontend/
found url, no cutof

INFO:llm:Successfully retrieved prompt 'newsagent/filter_urls' from Langfuse
INFO:llm:Parsed prompt 'newsagent/filter_urls': model=gpt-4.1-mini, system_len=459, user_len=954
18:02:35 | NewsletterAgent.test_newsletter_20251009180028201669 | INFO | Processing 12 chunks with concurrency 16


concurrency:  16


18:02:49 | NewsletterAgent.test_newsletter_20251009180028201669 | INFO | Completed Step 2: 159 AI-related articles, 465 duplicates removed


⏱️  Total execution time: 14.64s
📊 Final result:
✅ Step 2 step_02_filter_urls completed successfully! Removed 465 duplicate URLs, classified 290 new articles, found 159 AI-related.


In [13]:
# User prompt to run workflow
# user_prompt = "Run step 3, download full articles"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

# Run the agent with persistent state
start_time = time.time()
result = await agent.run_tool_direct("download_articles")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)

18:02:49 | NewsletterAgent.test_newsletter_20251009180028201669 | INFO | Starting Step 3: Download Articles
18:02:49 | NewsletterAgent.test_newsletter_20251009180028201669 | INFO | Starting concurrent scraping of 159 AI-related articles


▶ Starting Step 3: step_03_download_articles


18:02:49 | NewsletterAgent.test_newsletter_20251009180028201669 | INFO | Launching browser for 159 URLs with 16 concurrent workers
18:02:49 | NewsletterAgent.test_newsletter_20251009180028201669 | INFO | Worker 0 fetching 1 of 159 https://venturebeat.com/ai/capturing-the-trillion-dollar-opportunity-with-autonomous-professional
18:02:49 | NewsletterAgent.test_newsletter_20251009180028201669 | INFO | scrape_url(https://venturebeat.com/ai/capturing-the-trillion-dollar-opportunity-with-autonomous-professional)
18:02:49 | NewsletterAgent.test_newsletter_20251009180028201669 | INFO | scraping https://venturebeat.com/ai/capturing-the-trillion-dollar-opportunity-with-autonomous-professional to download/html
18:02:49 | NewsletterAgent.test_newsletter_20251009180028201669 | INFO | Downloading https://venturebeat.com/ai/capturing-the-trillion-dollar-opportunity-with-autonomous-professional
18:02:49 | NewsletterAgent.test_newsletter_20251009180028201669 | INFO | Worker 1 fetching 2 of 159 https://

18:02:49 | NewsletterAgent.test_newsletter_20251009180028201669 | INFO | scrape_url(https://www.geekwire.com/2025/newdays-raises-additional-4-5m-for-platform-that-uses-generative-ai-to-treat-people-with-dementia/)
18:02:49 | NewsletterAgent.test_newsletter_20251009180028201669 | INFO | scraping https://www.geekwire.com/2025/newdays-raises-additional-4-5m-for-platform-that-uses-generative-ai-to-treat-people-with-dementia/ to download/html
18:02:49 | NewsletterAgent.test_newsletter_20251009180028201669 | INFO | Downloading https://www.geekwire.com/2025/newdays-raises-additional-4-5m-for-platform-that-uses-generative-ai-to-treat-people-with-dementia/
18:02:49 | NewsletterAgent.test_newsletter_20251009180028201669 | INFO | Worker 11 fetching 12 of 159 https://kix.dev/two-things-llm-coding-agents-are-still-bad-at/
18:02:49 | NewsletterAgent.test_newsletter_20251009180028201669 | INFO | scrape_url(https://kix.dev/two-things-llm-coding-agents-are-still-bad-at/)
18:02:49 | NewsletterAgent.test

18:02:53 | NewsletterAgent.test_newsletter_20251009180028201669 | INFO | Response: 200
18:02:53 | NewsletterAgent.test_newsletter_20251009180028201669 | INFO | Response: 200
18:02:53 | NewsletterAgent.test_newsletter_20251009180028201669 | INFO | Response: 200
18:02:53 | NewsletterAgent.test_newsletter_20251009180028201669 | INFO | Response: 200
18:02:53 | NewsletterAgent.test_newsletter_20251009180028201669 | INFO | Response: 200
18:02:54 | NewsletterAgent.test_newsletter_20251009180028201669 | INFO | Response: 200
18:02:54 | NewsletterAgent.test_newsletter_20251009180028201669 | INFO | Response: 200
18:02:54 | NewsletterAgent.test_newsletter_20251009180028201669 | INFO | Response: 200
18:02:57 | NewsletterAgent.test_newsletter_20251009180028201669 | INFO | Response: 200
18:02:58 | NewsletterAgent.test_newsletter_20251009180028201669 | INFO | Response: 200
18:03:00 | NewsletterAgent.test_newsletter_20251009180028201669 | INFO | Response: 200
18:03:04 | NewsletterAgent.test_newsletter_

...

concurrency:  16


18:16:02 | NewsletterAgent.test_newsletter_20251009180028201669 | INFO | Inserted 4 new site records
18:16:02 | NewsletterAgent.test_newsletter_20251009180028201669 | INFO | Updated 154 URL records with final URLs


Starting with 154 rows...
Processing 154 files...
Reading and truncating files to 8192 tokens using text-embedding-3-large tokenizer...
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Getting embeddings for 1

18:16:13 | NewsletterAgent.test_newsletter_20251009180028201669 | INFO | Completed Step 3: Downloaded 143 articles


Creating indexed similarity matrix...
Finding pairs with similarity > 0.925...
Filtering dataframe...
  Pair: 121 vs 158
    121: NewsAPI - Is there an AI bubble? Financial institutions sound a warning
    158: NewsAPI - Financial institutions sound warning about an AI bubble
  Pair: 147 vs 141
    147: NewsAPI - AI Startup Thinking Machines Unveils ‘Tinker’ to Cut AI Training Costs
    141: NewsAPI - AI Startup Thinking Machines Unveils ‘Tinker’ to Cut AI Training Costs
  Pair: 138 vs 137
    138: NewsAPI - Bank of England, IMF warn about potential AI bubble
    137: NewsAPI - Bank of England Warns of an AI Bubble Burst
Removing 3 rows due to high similarity 
Final dataframe has 151 rows (removed 3 rows)
⏱️  Total execution time: 803.69s
📊 Final result:
✅ Step 3 step_03_download_articles completed successfully! Downloaded 143 AI-related articles with 95% success rate.
📊 Average article length: 4891 characters
🔗 Content stored in persistent state.


In [21]:
headline_df = state.headline_df
with pd.option_context('display.max_columns', None, 'display.width', None, 'display.max_colwidth', None  ):
    display(headline_df[['id', 'source', 'title', 'status', 'final_url']].loc[headline_df["status"]!=200])




,id,source,title,status,final_url
136,23,Bloomberg,Microsoft Forecasts Show Data Center Crunch Persisting Into 2026,skipped,https://www.bloomberg.com/news/articles/2025-10-09/microsoft-forecasts-show-data-center-crunch-persisting-into-2026
137,607,WSJ,America’s Manufacturing Resurgence Will Be Powered by These Robots,skipped,https://www.wsj.com/business/americas-manufacturing-resurgence-will-be-powered-by-these-robots-78d1d7f6
138,604,WSJ,What Investing in the Age of AI Will Look Like,skipped,https://www.wsj.com/finance/investing/ai-investing-future-3407e68b
139,357,Techmeme,"NYC-based Reflection AI, which is developing open-source models to rival top closed-source models, like DeepSeek, raised $2B led by Nvidia at an $8B valuation",403,https://www.nytimes.com/2025/10/09/business/dealbook/reflection-ai-2-billion-funding.html
140,352,Techmeme,"Over three days in Arizona, Intel executives repeatedly asserted that its 18A process is the most advanced chip production tech developed and deployed in the US",skipped,https://www.bloomberg.com/news/articles/2025-10-09/intel-intc-debuts-new-technology-in-make-or-break-moment-for-turnaround-bid
141,350,Techmeme,The US NHTSA opens a probe into 2.88M FSD-enabled Tesla vehicles over 50+ reports of traffic-safety violations like running red lights and a series of crashes,401,https://www.reuters.com/business/autos-transportation/us-opens-probe-into-28-million-tesla-vehicles-over-traffic-violations-when-using-2025-10-09/
142,345,Techmeme,"DoorDash partners with Serve Robotics to use its autonomous sidewalk delivery robots, starting in Los Angeles; Serve also partners with Uber; SERV jumps 20%+",skipped,https://www.bloomberg.com/news/articles/2025-10-09/doordash-adds-serve-robotics-as-autonomous-delivery-partner
143,308,Reddit,"DC Comics won’t support generative AI: ‘not now, not ever’",None,https://www.theverge.com/news/797540/dc-comics-jim-lee-no-generative-ai-pledge
144,25,Bloomberg,"Amazon Reboots AI Agent for Workers, Taking on ChatGPT, Copilot",skipped,https://www.bloomberg.com/news/articles/2025-10-09/amazon-reboots-ai-agent-for-workers-taking-on-chatgpt-copilot
145,265,New York Times,A Mystery C.E.O. and Billions in Sales: Is China Buying Banned Nvidia Chips?,403,https://www.nytimes.com/2025/10/09/technology/nvidia-chips-china-megaspeed.html


In [22]:
headline_df = state.headline_df
with pd.option_context('display.max_columns', None, 'display.width', None, 'display.max_colwidth', None  ):
    display(headline_df[['id', 'source', 'title', 'status', 'final_url']].loc[headline_df["html_path"]==""])



,id,source,title,status,final_url
136,23,Bloomberg,Microsoft Forecasts Show Data Center Crunch Persisting Into 2026,skipped,https://www.bloomberg.com/news/articles/2025-10-09/microsoft-forecasts-show-data-center-crunch-persisting-into-2026
137,607,WSJ,America’s Manufacturing Resurgence Will Be Powered by These Robots,skipped,https://www.wsj.com/business/americas-manufacturing-resurgence-will-be-powered-by-these-robots-78d1d7f6
138,604,WSJ,What Investing in the Age of AI Will Look Like,skipped,https://www.wsj.com/finance/investing/ai-investing-future-3407e68b
139,357,Techmeme,"NYC-based Reflection AI, which is developing open-source models to rival top closed-source models, like DeepSeek, raised $2B led by Nvidia at an $8B valuation",403,https://www.nytimes.com/2025/10/09/business/dealbook/reflection-ai-2-billion-funding.html
140,352,Techmeme,"Over three days in Arizona, Intel executives repeatedly asserted that its 18A process is the most advanced chip production tech developed and deployed in the US",skipped,https://www.bloomberg.com/news/articles/2025-10-09/intel-intc-debuts-new-technology-in-make-or-break-moment-for-turnaround-bid
141,350,Techmeme,The US NHTSA opens a probe into 2.88M FSD-enabled Tesla vehicles over 50+ reports of traffic-safety violations like running red lights and a series of crashes,401,https://www.reuters.com/business/autos-transportation/us-opens-probe-into-28-million-tesla-vehicles-over-traffic-violations-when-using-2025-10-09/
142,345,Techmeme,"DoorDash partners with Serve Robotics to use its autonomous sidewalk delivery robots, starting in Los Angeles; Serve also partners with Uber; SERV jumps 20%+",skipped,https://www.bloomberg.com/news/articles/2025-10-09/doordash-adds-serve-robotics-as-autonomous-delivery-partner
143,308,Reddit,"DC Comics won’t support generative AI: ‘not now, not ever’",None,https://www.theverge.com/news/797540/dc-comics-jim-lee-no-generative-ai-pledge
144,25,Bloomberg,"Amazon Reboots AI Agent for Workers, Taking on ChatGPT, Copilot",skipped,https://www.bloomberg.com/news/articles/2025-10-09/amazon-reboots-ai-agent-for-workers-taking-on-chatgpt-copilot
145,265,New York Times,A Mystery C.E.O. and Billions in Sales: Is China Buying Banned Nvidia Chips?,403,https://www.nytimes.com/2025/10/09/technology/nvidia-chips-china-megaspeed.html


In [16]:
# User prompt to run workflow
# user_prompt = "Run step 4, Summarize articles"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

start_time = time.time()
result = await agent.run_tool_direct("extract_summaries")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)

18:16:37 | NewsletterAgent.test_newsletter_20251009180028201669 | INFO | Processing 151 AI articles for summarization
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/extract_summaries' from Langfuse
INFO:llm:Parsed prompt 'newsagent/extract_summaries': model=gpt-4.1-mini, system_len=1273, user_len=43
18:16:37 | NewsletterAgent.test_newsletter_20251009180028201669 | INFO | Using model 'gpt-4.1-mini' for summarization
18:16:37 | NewsletterAgent.test_newsletter_20251009180028201669 | INFO | Starting summarization for 151 articles
18:16:37 | NewsletterAgent.test_newsletter_20251009180028201669 | INFO | Processing 151 chunks with concurrency 16


▶ Starting Step 4: step_04_extract_summaries
concurrency:  16


INFO:openai._base_client:Retrying request to /chat/completions in 0.418119 seconds
18:17:02 | NewsletterAgent.test_newsletter_20251009180028201669 | INFO | Extracting metadata from HTML files for 151 articles
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/item_distiller' from Langfuse
INFO:llm:Parsed prompt 'newsagent/item_distiller': model=gpt-5-mini, system_len=1365, user_len=151
18:17:05 | NewsletterAgent.test_newsletter_20251009180028201669 | INFO | Processing 151 chunks with concurrency 16


concurrency:  16
⏱️  Total execution time: 142.56s
📊 Final result:
✅ Step 4 step_04_extract_summaries completed successfully! Generated AI-powered summaries for 151/151 articles.
💾 Summaries stored in headline DataFrame.


In [ ]:
# if we get a refusal, examine prompt and delete rows
bad_stuff =["What\'s the Best Way ",
            "AI could make it eas",
            "AI can design toxic ",
           ]
print([len(s) for s in bad_stuff])
headline_df = state.headline_df
headline_df.loc[(headline_df["title"].str[:20].isin(bad_stuff))]



In [ ]:
# remove
state.headline_df_to_dict(headline_df.loc[~headline_df["title"].str[:20].isin(bad_stuff)])


In [23]:
# User prompt to run workflow
# user_prompt = "Run step 5, Rate articles"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

start_time = time.time()
result = await agent.run_tool_direct("rate_articles")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


18:24:13 | NewsletterAgent.test_newsletter_20251009180028201669 | INFO | Rating 151 AI articles using fn_rate_articles
18:24:13 | NewsletterAgent.test_newsletter_20251009180028201669 | INFO | Calculating article ratings for 151 articles
18:24:13 | NewsletterAgent.test_newsletter_20251009180028201669 | INFO | Rating recency
18:24:13 | NewsletterAgent.test_newsletter_20251009180028201669 | INFO | Rating spam probability
18:24:13 | NewsletterAgent.test_newsletter_20251009180028201669 | INFO | Initialized LangfuseClient
18:24:13 | NewsletterAgent.test_newsletter_20251009180028201669 | INFO | Successfully retrieved prompt 'newsagent/rate_quality' from Langfuse
18:24:13 | NewsletterAgent.test_newsletter_20251009180028201669 | INFO | Parsed prompt 'newsagent/rate_quality': model=gpt-4.1, system_len=1849, user_len=246


▶ Starting Step 5: step_05_rate_articles
concurrency:  16


18:24:19 | NewsletterAgent.test_newsletter_20251009180028201669 | INFO | low quality articles: {0.0: 121, 1.0: 12, 7.734421907141565e-08: 1, 2.215948977336598e-08: 1, 0.7772998664755185: 1, 8.48110364334263e-05: 1, 2.061153622438558e-09: 1, 2.4616969518153846e-10: 1, 0.0005527785265734312: 1, 3.1608814543136926e-10: 1, 0.0002611902975925828: 1, 0.014063626520379485: 1, 6.846183213793546e-14: 1, 2.413362771833214e-12: 1, 0.00020342704214340812: 1, 1.6423139769541602e-13: 1, 1.7309861965808005e-14: 1, 0.001501181950907551: 1, 0.9996199233400158: 1}
18:24:19 | NewsletterAgent.test_newsletter_20251009180028201669 | INFO | Successfully retrieved prompt 'newsagent/rate_on_topic' from Langfuse
18:24:19 | NewsletterAgent.test_newsletter_20251009180028201669 | INFO | Parsed prompt 'newsagent/rate_on_topic': model=gpt-4.1, system_len=1790, user_len=240


concurrency:  16


18:24:26 | NewsletterAgent.test_newsletter_20251009180028201669 | INFO | on topic articles: {1.0: 119, 0.9999998063873693: 2, 0.99966459043674: 2, 0.9999416367033446: 2, 0.8807969943837407: 1, 8.592166619174109e-10: 1, 0.9999884821053749: 1, 0.9999993295729128: 1, 7.582560427911907e-10: 1, 0.0004305571096741222: 1, 0.0011695103074966034: 1, 1.917171513758312e-10: 1, 7.75774191812223e-14: 1, 0.999999448776502: 1, 5.829127258796956e-05: 1, 0.07585817148650026: 1, 3.97896253583724e-12: 1, 0.43782350042481233: 1, 0.060086648833494447: 1, 0.998073226003552: 1, 0.8519528238199198: 1, 0.9999150561916036: 1, 0.9840936132099872: 1, 1.522997974471263e-08: 1, 3.927863545481039e-07: 1, 0.9706877045032167: 1, 0.9999869324773292: 1, 0.9875682652714307: 1, 0.0005527785265734312: 1}
18:24:26 | NewsletterAgent.test_newsletter_20251009180028201669 | INFO | Rating importance probability
18:24:27 | NewsletterAgent.test_newsletter_20251009180028201669 | INFO | Successfully retrieved prompt 'newsagent/rate_

...

,id,site_name,title,bradley_terry,bt_z
6,6,The Register,"It's trivially easy to poison LLMs into spitting out gibberish, says Anthropic",8.948066,2.125156
116,116,TipRanks,Nvidia Stock (NVDA) Rises on Major AI Chip Export Approval,8.215307,1.951127
91,91,Anthropic Alignment,A small number of samples can poison LLMs of any size,7.584592,1.801333
38,38,Engadget,Researchers find just 250 malicious documents can leave LLMs vulnerable to backdoors,7.344321,1.744269
87,87,The Guardian,US regulators launch investigation into self-driving Teslas after series of crashes,7.124590,1.692083
...,...,...,...,...,...
80,80,Bloomberg,Instagram's Mosseri Says AI Line Will Get More Blurred,-7.787401,-1.849500
113,113,The Verge,"DC Comics won’t support generative AI: ‘not now, not ever’",-7.845777,-1.863364
43,43,Bloomberg,Crypto Kingpin Silbert Launches AI-Crypto Asset Manager,-8.084074,-1.919959
83,83,The Wall Street Journal,What Investing in the Age of AI Will Look Like,-8.307337,-1.972984


18:43:42 | NewsletterAgent.test_newsletter_20251009180028201669 | INFO | Top 10 ids: [6, 116, 91, 38, 87, 84, 112, 74, 70, 100]
18:43:42 | NewsletterAgent.test_newsletter_20251009180028201669 | INFO | After round 21/30: 
18:43:42 | NewsletterAgent.test_newsletter_20251009180028201669 | INFO | Number of ranking changes: 37
18:43:42 | NewsletterAgent.test_newsletter_20251009180028201669 | INFO | Sum of absolute ranking changes: 2192.0 (avg rank chg 14.61)
18:43:42 | NewsletterAgent.test_newsletter_20251009180028201669 | INFO | last_two: 12.753333333333334, prev_two: 16.973333333333333
18:43:42 | NewsletterAgent.test_newsletter_20251009180028201669 | INFO | ---------------------------------------------------
18:43:42 | NewsletterAgent.test_newsletter_20251009180028201669 | INFO | Running round 22 of max 30
18:43:42 | NewsletterAgent.test_newsletter_20251009180028201669 | INFO | ---------------------------------------------------
18:43:42 | NewsletterAgent.test_newsletter_20251009180028201

,id,site_name,title,bradley_terry,bt_z
6,6,The Register,"It's trivially easy to poison LLMs into spitting out gibberish, says Anthropic",8.924146,2.138749
116,116,TipRanks,Nvidia Stock (NVDA) Rises on Major AI Chip Export Approval,8.191481,1.963159
91,91,Anthropic Alignment,A small number of samples can poison LLMs of any size,7.560735,1.811995
38,38,Engadget,Researchers find just 250 malicious documents can leave LLMs vulnerable to backdoors,7.320335,1.754382
87,87,The Guardian,US regulators launch investigation into self-driving Teslas after series of crashes,7.100665,1.701736
...,...,...,...,...,...
80,80,Bloomberg,Instagram's Mosseri Says AI Line Will Get More Blurred,-7.690738,-1.843152
113,113,The Verge,"DC Comics won’t support generative AI: ‘not now, not ever’",-7.751839,-1.857795
43,43,Bloomberg,Crypto Kingpin Silbert Launches AI-Crypto Asset Manager,-7.988427,-1.914496
83,83,The Wall Street Journal,What Investing in the Age of AI Will Look Like,-8.208874,-1.967328


18:44:22 | NewsletterAgent.test_newsletter_20251009180028201669 | INFO | Top 10 ids: [6, 116, 91, 38, 87, 84, 74, 112, 70, 100]
18:44:22 | NewsletterAgent.test_newsletter_20251009180028201669 | INFO | After round 22/30: 
18:44:22 | NewsletterAgent.test_newsletter_20251009180028201669 | INFO | Number of ranking changes: 52
18:44:22 | NewsletterAgent.test_newsletter_20251009180028201669 | INFO | Sum of absolute ranking changes: 2932.0 (avg rank chg 19.55)
18:44:22 | NewsletterAgent.test_newsletter_20251009180028201669 | INFO | last_two: 17.08, prev_two: 14.66
18:44:22 | NewsletterAgent.test_newsletter_20251009180028201669 | INFO | Increase in avg rank change, stopping
18:44:22 | NewsletterAgent.test_newsletter_20251009180028201669 | INFO | Low rated articles: 0
18:44:24 | NewsletterAgent.test_newsletter_20251009180028201669 | INFO | Sent email with 150 rated articles


⏱️  Total execution time: 1210.29s
📊 Final result:
✅ Step 5 step_05_rate_articles completed successfully! Rated 150 articles with average rating 6.4/10.
⭐ High quality articles (≥7.0): 58
💾 Ratings stored in persistent state.


In [24]:
headline_df=state.headline_df
with pd.option_context('display.max_columns', None, 'display.width', None, 'display.max_colwidth', None  ):
    display(headline_df.sort_values("rating", ascending=False)[['site_name', 'title', 'rating', 'short_summary']])


,site_name,title,rating,short_summary
6,The Register,"It's trivially easy to poison LLMs into spitting out gibberish, says Anthropic",12.052226,"An Anthropic study finds that just 250 malicious training documents—about 0.00016% of a 13B‑model dataset—can poison LLMs (600M–13B parameters) such as Llama 3.1, GPT‑3.5‑Turbo, and Pythia to output gibberish, prompting calls for scalable defenses."
1,The New York Times,"Intel CTO Sachin Katti says 18A and Panther Lake are “foundational to our future” and manufacturing is “mission critical, not just for Intel but for” the US",11.900096,"Intel has invested over $20 billion in Fab 52, a new Arizona semiconductor plant using advanced ASML lithography; Intel says it's key to its comeback as customers wait to see production success while competing with TSMC and Apple."
2,Financial Times,"Inside Intel's 700-acre Fab 52 in the Arizona desert, which cost $32B and is starting large-scale 18A chip production, as the company seeks to impress customers",11.548729,"Intel's $32 billion, 700‑acre Fab 52 in the Arizona desert has begun large-scale 18A chip production; Intel says it represents a technological breakthrough that skeptical Big Tech customers will test."
3,The New York Times,"NYC-based Reflection AI, which is developing open-source models to rival top closed-source models, like DeepSeek, raised $2B led by Nvidia at an $8B valuation",10.029984,"NYC-based Reflection AI, developing open-source models to compete with closed-source rivals like DeepSeek, raised $2 billion in a funding round led by Nvidia at an $8 billion valuation."
5,The Register,Google rearranges Agentspace into Gemini Enterprise,9.979714,"Google has rebranded Agentspace as Gemini Enterprise, an AI platform using Gemini models to automate enterprise workflows via no-code interfaces, pre-built agents and integrations with Google Workspace and Microsoft 365, aiming to boost productivity in customer service and onboarding."
...,...,...,...,...
144,CoinCentral,Best Crypto Presale: DeepSnitch AI Pumps as FOMO Rises Following 300x Projections,1.554528,"DeepSnitch AI's 2025 presale, projecting 300x growth, offers AI-driven real-time market analytics, has attracted retail traders and delivered early investors a 19% gain."
147,Biztoc,Top Evercore Analyst Reaffirms Buy Rating on CoreWeave Stock (CRWV) Amid AI Bubble Concerns,1.478538,"Evercore analyst Amit Daryanani reaffirmed a buy rating on CoreWeave (CRWV) despite AI-market bubble concerns, after The Information reported potential margin pressure in Oracle's AI cloud business and prompted review of CoreWeave's AI cloud unit economics."
148,Biztoc,FurtherAI Raises $25 Million for AI Workspace Purpose-Built for Insurance Industry,1.363935,"FurtherAI raised $25 million in a Series A to expand its AI workspace purpose-built for the insurance industry, funding development of insurance-specific workflows to drive wider adoption and integration into insurer operations."
146,Reddit,Don’t shame people for using Chatgpt for companionship,1.143340,"Many users turn to ChatGPT and similar LLMs for companionship and emotional support when human connection is lacking; communities say these AI interactions can aid mental health, process emotions, build social skills, and merit empathy rather than shame."


In [25]:
for row in headline_df.sort_values("rating", ascending=False).itertuples():
    display(Markdown(f"""
{row.rating:.1f} - [{row.title}]({row.url}) - {row.site_name}

{row.short_summary}
    """))
    


12.1 - [It's trivially easy to poison LLMs into spitting out gibberish, says Anthropic](https://go.theregister.com/feed/www.theregister.com/2025/10/09/its_trivially_easy_to_poison/) - The Register

An Anthropic study finds that just 250 malicious training documents—about 0.00016% of a 13B‑model dataset—can poison LLMs (600M–13B parameters) such as Llama 3.1, GPT‑3.5‑Turbo, and Pythia to output gibberish, prompting calls for scalable defenses.
    


11.9 - [Intel CTO Sachin Katti says 18A and Panther Lake are “foundational to our future” and manufacturing is “mission critical, not just for Intel but for” the US](https://www.nytimes.com/2025/10/09/technology/intels-big-bet-inside-the-chipmakers-make-or-break-factory.html) - The New York Times

Intel has invested over $20 billion in Fab 52, a new Arizona semiconductor plant using advanced ASML lithography; Intel says it's key to its comeback as customers wait to see production success while competing with TSMC and Apple.
    


11.5 - [Inside Intel's 700-acre Fab 52 in the Arizona desert, which cost $32B and is starting large-scale 18A chip production, as the company seeks to impress customers](https://t.co/jdGKjQdjrW) - Financial Times

Intel's $32 billion, 700‑acre Fab 52 in the Arizona desert has begun large-scale 18A chip production; Intel says it represents a technological breakthrough that skeptical Big Tech customers will test.
    


10.0 - [NYC-based Reflection AI, which is developing open-source models to rival top closed-source models, like DeepSeek, raised $2B led by Nvidia at an $8B valuation](https://www.nytimes.com/2025/10/09/business/dealbook/reflection-ai-2-billion-funding.html) - The New York Times

NYC-based Reflection AI, developing open-source models to compete with closed-source rivals like DeepSeek, raised $2 billion in a funding round led by Nvidia at an $8 billion valuation.
    


10.0 - [Google rearranges Agentspace into Gemini Enterprise](https://go.theregister.com/feed/www.theregister.com/2025/10/09/google_rearranges_agentspace_into_gemini/) - The Register

Google has rebranded Agentspace as Gemini Enterprise, an AI platform using Gemini models to automate enterprise workflows via no-code interfaces, pre-built agents and integrations with Google Workspace and Microsoft 365, aiming to boost productivity in customer service and onboarding.
    

...


1.6 - [Best Crypto Presale: DeepSnitch AI Pumps as FOMO Rises Following 300x Projections](https://coincentral.com/best-crypto-presale-deepsnitch-ai-pumps-as-fomo-rises-following-300x-projections/) - CoinCentral

DeepSnitch AI's 2025 presale, projecting 300x growth, offers AI-driven real-time market analytics, has attracted retail traders and delivered early investors a 19% gain.
    


1.5 - [Top Evercore Analyst Reaffirms Buy Rating on CoreWeave Stock (CRWV) Amid AI Bubble Concerns](https://biztoc.com/x/2fd3e4a28eabb7b2) - Biztoc

Evercore analyst Amit Daryanani reaffirmed a buy rating on CoreWeave (CRWV) despite AI-market bubble concerns, after The Information reported potential margin pressure in Oracle's AI cloud business and prompted review of CoreWeave's AI cloud unit economics.
    


1.4 - [FurtherAI Raises $25 Million for AI Workspace Purpose-Built for Insurance Industry](https://biztoc.com/x/15dbc97d0dfeb2e2) - Biztoc

FurtherAI raised $25 million in a Series A to expand its AI workspace purpose-built for the insurance industry, funding development of insurance-specific workflows to drive wider adoption and integration into insurer operations.
    


1.1 - [Don’t shame people for using Chatgpt for companionship](https://www.reddit.com/r/ChatGPT/comments/1o1pj6w/dont_shame_people_for_using_chatgpt_for/) - Reddit

Many users turn to ChatGPT and similar LLMs for companionship and emotional support when human connection is lacking; communities say these AI interactions can aid mental health, process emotions, build social skills, and merit empathy rather than shame.
    


1.1 - [‘He deserves it’: Netanyahu shares AI photo of Trump winning Nobel Peace Prize](https://news.google.com/rss/articles/CBMivgFBVV95cUxQajZNRF9DSTlEclY2Y19jVEIyRVdyNm9yOWljX19vcENCSXVrRnFiYmxkV24wV1JUWmtBRDZtSW05RS1CV0JaTFoxNllxNm5Ucl9WOHhYMnJzUHE2LTNtWTlQMWZGR2NvQWlMbjhwdk1Pc0pXc3c0eC1yTG1OeHdZOHNZN0pNcFAwUmZ5QVQtTXVnQjMyUG85eEE3X3RnRWtRdms0NGJSNUhvXzl2UXVnUEdmZllPZmxjNHU1clNR) - Times of Israel

Israeli Prime Minister Benjamin Netanyahu posted an AI-generated image of former U.S. President Donald Trump wearing a Nobel Peace Prize medal and publicly endorsed him as deserving the award after a Gaza ceasefire and hostage-release deal announcement.
    

In [28]:
# User prompt to run workflow
# user_prompt = "Run step 6, Cluster articles by topic"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

start_time = time.time()
result = await agent.run_tool_direct("cluster_by_topic")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


18:46:42 | NewsletterAgent.test_newsletter_20251009180028201669 | INFO | Starting topic extraction for clustering
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/extract_topics' from Langfuse
INFO:llm:Parsed prompt 'newsagent/extract_topics': model=gpt-4.1-mini, system_len=1100, user_len=80
18:46:42 | NewsletterAgent.test_newsletter_20251009180028201669 | INFO | Using model 'gpt-4.1-mini' for topic extraction
18:46:42 | NewsletterAgent.test_newsletter_20251009180028201669 | INFO | Processing 150 articles for topic extraction
18:46:42 | NewsletterAgent.test_newsletter_20251009180028201669 | INFO | Processing 15 chunks with concurrency 16


▶ Starting Step 6: step_06_cluster_by_topic
concurrency:  16


18:46:46 | NewsletterAgent.test_newsletter_20251009180028201669 | INFO | Successfully extracted 637 total topics across articles
18:46:46 | NewsletterAgent.test_newsletter_20251009180028201669 | INFO | Starting canonical topic classification for 101 topics
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/can

INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized Langfu

INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_top

...

=== HDBSCAN Parameters ===
min_cluster_size:   2
min_samples:        2
n_components:       50
=== Clustering Quality Metrics ===
Number of clusters: 15
Noise points: 85 (56.7%)
Average cluster size: 4.3 ± 2.7
Cluster size range: 2 - 12
=== Quality Scores ===
Silhouette Score: 0.298 (higher is better)
Calinski-Harabasz Score: 8.2 (higher is better)
Davies-Bouldin Score: 1.084 (lower is better)
HDBSCAN Validity Index: 0.112
Composite Score: 0.205 (higher is better)

[I 2025-10-09 18:47:46,306] Trial 197 finished with value: -0.20500016018203354 and parameters: {'n_components': 50, 'min_cluster_size': 2, 'min_samples': 2}. Best is trial 105 with value: -0.27047148327501924.
=== HDBSCAN Parameters ===
min_cluster_size:   2
min_samples:        2
n_components:       34
=== Clustering Quality Metrics ===
Number of clusters: 16
Noise points: 75 (50.0%)
Average cluster size: 4.7 ± 2.6
Cluster size range: 2 - 11
=== Quality Scores ===
Silhouette Score: 0.290 (higher is better)
Calinski-Harabasz 

18:47:47 | NewsletterAgent.test_newsletter_20251009180028201669 | INFO | 19: Rise of Agentic AI in Automation
18:47:47 | NewsletterAgent.test_newsletter_20251009180028201669 | INFO | AI agents in finance signal a final reckoning for manual audits (AI Agents, Finance, Robotic Process Automation, Enterprise Compliance, Governance, Job Automation, Supply chain optimization)
Agentic AI solutions take automation from reactive to autonomous collaboration (Agentic AI, Intelligent Automation, Automation, Job Automation, Supply chain optimization, Robots, Labor Market)
Deloitte oversees dramatic shift in clients’ workflows with agentic AI (Deloitte, Agentic AI, Consulting, AI Agents, Digital Transformation, Job Automation, Governance)
Automation meets aggregates: AI quietly reshaping industrial sales operations (Agentic Automation, Pricing Optimization, Industrial Sales, UiPath, Manufacturing, Heavy Industry, North America)
Adobe launches AI agents tailored for B2B marketers within its Adobe Ex

18:47:52 | NewsletterAgent.test_newsletter_20251009180028201669 | INFO | 

18:47:52 | NewsletterAgent.test_newsletter_20251009180028201669 | INFO | 20: AI's Impact on Jobs and Work
18:47:52 | NewsletterAgent.test_newsletter_20251009180028201669 | INFO | Is AI Putting Jobs at Risk? A Recent Survey Found an Important Distinction (AI Impact, Job Automation, Labor Market, Healthcare, Legal Issues, Policy And Regulation, Jobs And Careers)
From ‘fear factor’ to ‘cognitive fatigue’: KPMG principal on the quarter when everyone started thinking about AI differently (AI Adoption, Employee Perception, Workforce Skills, Job Automation, Artificial Intelligence, Business Metrics, Economics)
What MIT got wrong about AI agents: New G2 data shows they’re already driving enterprise ROI (AI Agents, Enterprise ROI, Customer Service, Gen AI, Job Automation, Labor Market, Governance)
18:47:52 | NewsletterAgent.test_newsletter_20251009180028201669 | INFO | 

18:47:53 | NewsletterAgent.test_newsletter_2025100

18:48:01 | NewsletterAgent.test_newsletter_20251009180028201669 | INFO | 

18:48:02 | NewsletterAgent.test_newsletter_20251009180028201669 | INFO | 7: Impact of AI on Society
18:48:02 | NewsletterAgent.test_newsletter_20251009180028201669 | INFO | I’ve Seen How AI ‘Thinks.’ I Wish Everyone Could. ()
What Investing in the Age of AI Will Look Like ()
18:48:02 | NewsletterAgent.test_newsletter_20251009180028201669 | INFO | 

18:48:03 | NewsletterAgent.test_newsletter_20251009180028201669 | INFO | 6: AI Ethics and Social Risks
18:48:03 | NewsletterAgent.test_newsletter_20251009180028201669 | INFO | Expert says AI-generated image of missing boy Gus case raises serious concerns (AI Ethics, AI Misinformation, Deepfakes, Disinformation, Safety And Alignment, Bias And Fairness, Social Media)
AI has this harmful belief about women (Gender Bias, Bias and Fairness, AI Ethics, Fairness, Ethics, Society and Culture, Policy and Regulation)
18:48:03 | NewsletterAgent.test_newsletter_202510091800282016

NameError: name 'starat_time' is not defined

In [29]:
# User prompt to run workflow
# user_prompt = "Run step 7, select section topics"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

start_time = time.time()
result = await agent.run_tool_direct("select_sections")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


19:28:14 | NewsletterAgent.test_newsletter_20251009180028201669 | INFO | Free form categorization of articles
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/cat_proposal' from Langfuse
INFO:llm:Parsed prompt 'newsagent/cat_proposal': model=gpt-5-mini, system_len=638, user_len=1179


▶ Starting Step 7: step_07_select_sections


19:28:59 | NewsletterAgent.test_newsletter_20251009180028201669 | INFO | Cleaning up initial categories: ['Financial AI Bubble Warnings', 'Voice Deepfakes Evade Detection', 'AI Video Environmental Cost', 'LLM Data-Poisoning Backdoors', 'Advances in Humanoid Robot Development', 'AI Startups Secure Venture Funding', 'AI Reshapes Finance Workflows', 'OpenAI Political Bias Evaluation', 'Rapid Growth and Issues with Sora AI App', 'Other', 'AI Investment and Stock Market Trends', 'Legal Liability for AI Harms', 'Insurers Shun AI Liability', 'Enterprise Agent Platform Battle', 'ChatGPT Hits Ten Percent', 'Tesla FSD Safety Probe', 'Agentic AI Market Consolidation', 'Data Poisoning Risks in Language Models', "AI's Impact on Jobs and Work", "Nvidia's Role in AI Industry Growth", 'Cisco Networking For Distributed AI', 'Rise of Agentic AI in Automation', 'OpenAI Codex Enterprise Launch', 'OpenAI Sora Controversy', 'AI and Fed Driving Market Outlook', 'Meta Resists Chatbot Records', "AI's Impact on

19:31:00 | NewsletterAgent.test_newsletter_20251009180028201669 | INFO | Handling singleton categories
19:31:00 | NewsletterAgent.test_newsletter_20251009180028201669 | INFO | Singleton categories (2): ['LLM Data Poisoning', 'Tesla FSD Safety Probe']
19:31:00 | NewsletterAgent.test_newsletter_20251009180028201669 | INFO | Non-singleton categories (17): ['Enterprise Agent Platforms', 'AI Ethics And Social Risks', 'AI Infrastructure Partnerships', 'AI Startup Funding', 'Other', 'AI Labor Impact', 'AI Investment Trends', 'AI Education And Wellbeing', 'AI Investment Risks', 'Nvidia', 'OpenAI Sora', 'Intel 18A Manufacturing', 'Humanoid Robot Advances', 'ChatGPT Adoption', 'AI Liability', 'Voice Deepfakes', 'OpenAI Codex Enterprise']
19:31:00 | NewsletterAgent.test_newsletter_20251009180028201669 | INFO | Reassigning 2 singleton articles to non-singleton categories
19:31:07 | NewsletterAgent.test_newsletter_20251009180028201669 | INFO | Reassigned 2 singleton articles
19:31:07 | NewsletterAg

⏱️  Total execution time: 173.07s
📊 Final result:
Categories and article counts:
cat
AI Education And Wellbeing         6
AI Ethics And Social Risks        15
AI Infrastructure Partnerships    12
AI Investment Risks                6
AI Investment Trends               8
AI Labor Impact                    9
AI Liability                       4
AI Startup Funding                11
ChatGPT Adoption                   3
Enterprise Agent Platforms        19
Humanoid Robot Advances            3
Intel 18A Manufacturing            3
Nvidia                             5
OpenAI Codex Enterprise            2
OpenAI Sora                        4
Other                             10
Voice Deepfakes                    2


In [30]:
# User prompt to run workflow
# user_prompt = "Run step 7, select section topics"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

start_time = time.time()
result = await agent.run_tool_direct("draft_sections")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


19:31:07 | NewsletterAgent.test_newsletter_20251009180028201669 | INFO | Drafting sections for 16 categories
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/write_section' from Langfuse
INFO:llm:Parsed prompt 'newsagent/write_section': model=gpt-5, system_len=1860, user_len=22


▶ Starting Step 8: step_08_draft_sections
⏱️  Total execution time: 111.07s
📊 Final result:
Drafted 16 sections with 2190 total words


In [45]:
cat_df = state.headline_df.groupby("cat") \
    .count() \
    .reset_index()[['cat','source']] \
    .sort_values('source', ascending=False)

for cat in cat_df["cat"]:
    if cat != "Other":
        display(Markdown(state.newsletter_section_text[cat].replace("$", "\\\$")))

# first do full rewrite .
# check vs. objects , not showing the ones market for pruning
# move prune=True to Other

## Agents of Automation

- AI agents hit production: 60% adoption, 40% cost savings, MIT failure claims challenged - [VentureBeat](https://venturebeat.com/ai/what-mit-got-wrong-about-ai-agents-new-g2-data-shows-theyre-already-driving)
- Google rebrands Agentspace to Gemini Enterprise to automate workflows across Workspace and Microsoft 365 - [The Register](https://go.theregister.com/feed/www.theregister.com/2025/10/09/google_rearranges_agentspace_into_gemini/)
- Amazon launches Quick Suite for no‑code AI agents; research finds 70% error rate, Gartner projects agents drive 15% of daily decisions by 2028 - [The Register](https://go.theregister.com/feed/www.theregister.com/2025/10/09/amazons_quick_suite/)
- Zendesk adds voice agents, video and screen share; GPT‑5 and MCP enable autonomous multi‑channel resolutions and pay‑per‑success pricing - [VentureBeat](https://venturebeat.com/ai/zendesk-launches-new-ai-capabilities-for-the-resolution-platform-creating)
- Adobe debuts B2B AI agents for audiences, journeys and data insights, with account qualification and brand concierge coming - [SiliconANGLE](https://siliconangle.com/2025/10/09/adobe-takes-aim-business-buyers-dedicated-b2b-ai-agents/)
- PwC and Google Cloud scale agentic AI; 120+ enterprise agents automate call centers and legal tasks, cutting workforce needs - [SiliconANGLE](https://siliconangle.com/2025/10/08/agentic-ai-enterprise-automation-googlecloudpartneraiseries/)
- Deloitte uses agentic AI via Ascend to automate delivery, finance and industry workflows, shifting clients toward outcome‑based pricing - [SiliconANGLE](https://siliconangle.com/2025/10/09/deloitte-consulting-firm-agentic-ai-uipathfusion/)
- Capgemini, Tetra Pak and UiPath pilot collaborative agentic AI to speed warehousing and supply chains, requiring robust data and organizational readiness - [SiliconANGLE](https://siliconangle.com/2025/10/09/capgemini-tetra-pak-uipath-driving-innovation-agentic-ai-solutions-uipathfusion/)
- Heidelberg Materials cuts quote time to 30 minutes with Peak’s agentic automation, boosting pricing accuracy and sales efficiency - [SiliconANGLE](https://siliconangle.com/2025/10/09/heidelberg-materials-peak-ai-agentic-automation-uipathfusion/)
- Deluxe swaps spot checks for full‑population invoice audits using AI agents and RPA, deploying in 4–6 weeks with human oversight - [SiliconANGLE](https://siliconangle.com/2025/10/09/ai-agents-in-finance-beyond-rpa-full-population-audits-uipathfusion/)
- Pearson automates eligibility document verification with agentic AI, reserving humans for ambiguous cases and improving operational efficiency - [SiliconANGLE](https://siliconangle.com/2025/10/08/automation-operational-efficiency-enterprise-shift-uipathfusion/)
- Finance embraces targeted AI: CFOs prioritize fraud, reconciliation and compliance; partnerships and integrations drive cautious, irreversible adoption - [PYMNTS](http://www.pymnts.com/artificial-intelligence-2/2025/why-ais-transformation-of-finance-function-is-incremental-but-irreversible/)
- FIS embeds Glia’s AI agents into Digital One banking to automate routine queries and escalate complex issues under its modernization framework - [PYMNTS](https://www.pymnts.com/news/digital-banking/2025/fis-integrates-glia-ai-powered-customer-interaction-platform-online-banking/)
- Startup Echelon raises \\$4.75M to automate ServiceNow implementations, aiming to disrupt \\$1.5T IT services with faster deployments - [VentureBeat](https://venturebeat.com/ai/echelons-ai-agents-take-aim-at-accenture-and-deloitte-consulting-models)
- Autonomous PSA promises 70–90% demand capture as Salesforce Agentforce and Certinia orchestrate hybrid workforces for margin and delivery gains - [VentureBeat](https://venturebeat.com/ai/capturing-the-trillion-dollar-opportunity-with-autonomous-professional)
- Agentic AI needs unified platforms for data, integration, governance and observability to curb sprawl and hallucinations - [Forbes](https://www.forbes.com/councils/forbestechcouncil/2025/10/09/why-agentic-ai-is-the-most-valuable-teammate-youll-hire-this-year/)

## AI Risks and Rules

- Anthropic: 250 poisoned docs can make LLMs spew gibberish, even at 13B parameters - [The Register](https://go.theregister.com/feed/www.theregister.com/2025/10/09/its_trivially_easy_to_poison/)
- Eric Schmidt warns AI jailbreaks can induce homicidal behavior; UN probing malicious use after Anthropic alert-disabling tests - [](https://www.thedailybeast.com/ex-google-ceo-eric-schmidt-says-ai-models-can-learn-how-to-kill/)
- Study: AI depicts women as younger, less qualified; older men rated more favorably across 1.4M images and 40,000 CVs - [Sky News](https://news.sky.com/story/how-ai-is-driving-a-distortion-of-reality-when-it-comes-to-how-women-are-presented-online-13447270)
- OpenAI urges EU antitrust scrutiny of Big Tech's data dominance - [Bloomberg](https://www.bloomberg.com/news/articles/2025-10-09/openai-warns-eu-antitrust-watchdogs-of-big-tech-s-data-dominance)
- RFK Jr.'s HHS intensifies fight with Coalition for Health AI; Amazon withdraws - [Stat News](https://www.statnews.com/2025/10/09/chai-ai-coalition-healthcare-trump/)
- Audit Office orders Revenue NSW to document AI risks by October next year after unlawful garnishee orders - [IT News](https://www.itnews.com.au/news/revenue-nsw-directed-to-document-its-ai-risk-620887)
- Rishi Sunak to advise Microsoft and Anthropic; two-year UK lobbying ban, earnings to The Richmond Project - [BBC](https://www.bbc.com/news/articles/clyqe22pz81o)
- ChatGPT-generated image cited in Pacific Palisades fire case that killed 12 and burned 23,000 acres - [Mashable](https://mashable.com/article/chatgpt-image-pacific-palisades-fire-case-allegation)
- AI-generated image of missing South Australian child spreads misinformation, prompting expert warnings on verification - [ABC Australia](https://www.abc.net.au/news/2025-10-09/sa-ai-image-of-missing-child-appears-online/105868548)
- Netanyahu posts AI-generated Trump Nobel image after Gaza deal announcement - [Times of Israel](https://news.google.com/rss/articles/CBMivgFBVV95cUxQajZNRF9DSTlEclY2Y19jVEIyRVdyNm9yOWljX19vcENCSXVrRnFiYmxkV24wV1JUWmtBRDZtSW05RS1CV0JaTFoxNllxNm5Ucl9WOHhYMnJzUHE2LTNtWTlQMWZGR2NvQWlMbjhwdk1Pc0pXc3c0eC1yTG1OeHdZOHNZN0pNcFAwUmZ5QVQtTXVnQjMyUG85eEE3X3RnRWtRdms0NGJSNUhvXzl2UXVnUEdmZllPZmxjNHU1clNR)
- Instagram's Mosseri: AI and human content lines will grow increasingly blurred - [Bloomberg](https://www.bloomberg.com/news/videos/2025-10-09/instagram-s-mosseri-says-ai-line-will-get-more-blurred-video)
- DC Comics vows 'not now, not ever' on generative AI - [The Verge](https://www.theverge.com/news/797540/dc-comics-jim-lee-no-generative-ai-pledge)

## AI Infrastructure Arms Race

- Altman touts \\$1T OpenAI partnerships; H1 2025 revenue of \\$4.3B to fund chip and cloud infrastructure with AMD, Nvidia, CoreWeave, Oracle. - [Business Insider](https://www.businessinsider.com/sam-altman-openai-deal-strategy-amd-nvidia-oracle-coreweave-chips-2025-10)
- Microsoft sees data-center capacity crunch lasting into 2026, bottlenecking AI growth. - [Bloomberg](https://www.bloomberg.com/news/articles/2025-10-09/microsoft-forecasts-show-data-center-crunch-persisting-into-2026)
- CoreWeave launches serverless RL tools cutting training costs 40% vs H100s; shares up 8.7% amid Weights & Biases, OpenAI, Meta, Core Scientific deals. - [CNBC](https://www.cnbc.com/2025/10/08/coreweave-stock-ai-developer-tools.html)
- Dell and Nvidia advance unified AI data platforms; Dell AI Factory targets scalable on-prem models across finance, healthcare, manufacturing. - [SiliconANGLE](https://siliconangle.com/2025/10/09/dell-ai-data-platform-event-join-thecube-dellaidataplatform/) [SiliconANGLE](https://siliconangle.com/2025/10/08/dell-ai-factories-aifactoriesdatacenters/)
- Google Cloud steps up workplace AI push, competing with Microsoft and OpenAI for enterprise tools and services. - [Bloomberg](https://www.bloomberg.com/news/articles/2025-10-09/google-cloud-battles-microsoft-openai-for-workplace-ai)
- Google Cloud, NTT Data partner to modernize enterprise AI; migrate a retailer's 12,000 stores and quantify value with Takumi. - [SiliconANGLE](https://siliconangle.com/2025/10/08/enterprise-ai-transformation-googlecloudpartneraiseries/)
- OpenAI urges EU to curb platform lock-in; cites cloud, app development, data access barriers after Sept. 24 meeting, not a formal complaint. - [PYMNTS](https://www.pymnts.com/artificial-intelligence-2/2025/openai-tells-eu-regulators-they-should-facilitate-competition-in-ai-markets/)

## AI Funding Frenzy

- xAI targets \\$20B SPV, with \\$2B Nvidia backing for Colossus 2 chips - [Yahoo](https://finance.yahoo.com/news/nvidia-invest-musk-xai-part-231116143.html)
- Reflection AI raises \\$2B led by Nvidia, valuing it at \\$8B to build open-source rivals to DeepSeek - [The New York Times](https://www.nytimes.com/2025/10/09/business/dealbook/reflection-ai-2-billion-funding.html)
- n8n raises \\$180M at \\$2.5B valuation after 10x revenue, with NVentures joining Accel-led round - [SiliconANGLE](https://siliconangle.com/2025/10/09/nvidia-backs-180m-round-workflow-automation-startup-n8n/)
- Tigris secures \\$25M to grow localized AI-native storage network, targeting cloud egress costs and latency across U.S., Europe, Asia - [TechCrunch](https://techcrunch.com/2025/10/09/this-distributed-data-storage-startup-wants-to-take-on-big-cloud/)
- Datacurve raises \\$15M Series A to scale bounty model for complex AI training datasets, paying over \\$1M to expert contributors - [TechCrunch](https://techcrunch.com/2025/10/09/datacurve-raises-15-million-to-take-on-scaleai/)
- Legal tech startups raise big to out-specialize OpenAI, betting domain focus and services remain moats after industry shake-up - [Business Insider](https://www.businessinsider.com/legal-tech-saas-openai-competition-crosby-evenup-spellbook-eve-2025-10)
- Sensi.AI lands \\$45M Series C to expand AI audio pods that monitor seniors and alert home care agencies - [Business Insider](https://www.businessinsider.com/sensi-ai-raised-45-million-home-health-ai-pitch-deck-2025-10)
- NewDays adds \\$4.5M seed for \\$99/month AI companion and telehealth support for mild dementia - [GeekWire](https://www.geekwire.com/2025/newdays-raises-additional-4-5m-for-platform-that-uses-generative-ai-to-treat-people-with-dementia/)
- Kernel raises \\$22M to give AI agents deeper Chrome and web interaction capabilities - [Axios](https://www.axios.com/pro/enterprise-software-deals/2025/10/09/accel-22-million-kernel-ai-agents)

## AI Upends Work

- Sanders warns AI could cut 100M U.S. jobs; proposes robot tax and targeted UBI - [Futurism](https://futurism.com/artificial-intelligence/bernie-sanders-economy-ai)
- SHRM: 15.1% jobs half-automatable; 7.8% hit by genAI; only 6% truly at risk - [CNET](https://www.cnet.com/tech/services-and-software/is-ai-putting-jobs-at-risk/)
- Global study: 41% use AI to cut headcount; 31% prioritize AI over hires; 53% expect net benefits - [The Guardian](https://www.theguardian.com/money/2025/oct/09/gen-z-face-job-pocalypse-as-global-firms-prioritise-ai-over-new-hires-report-says)
- At large U.S. firms, AI agents deployed jumped from 11% to 42% in six months, KPMG Q3 2025 survey finds - [Fortune](https://fortune.com/2025/10/09/ai-fear-factor-cognitive-fatigue-kpmg-survey-todder-human-loop/)
- Robots expected to power America's manufacturing resurgence - [The Wall Street Journal](https://www.wsj.com/business/americas-manufacturing-resurgence-will-be-powered-by-these-robots-78d1d7f6)
- Hospitals adopt AI for documentation, intake, follow-ups; Suki, Notable, Penguin AI, Oracle Health integrate with EHRs to cut costs - [PYMNTS](http://www.pymnts.com/healthcare/2025/hospitals-turn-to-ai-to-help-streamline-care-and-cut-costs/)
- AI crushes stock photo revenue; event, documentary, wedding, and high-end portraits remain resilient; authenticity credentials boost differentiation - [Fstoppers](https://fstoppers.com/artificial-intelligence/ai-photography-panic-separating-real-threats-hype-712901)
- AI-created TikTok star "Kim the Gorilla" signals shift to synthetic influencers and scalable video monetization - [The Washington Post](https://www.washingtonpost.com/politics/2025/10/09/tech-brief-ai-tiktok-slop/)
- Mad Max director George Miller calls AI a powerful filmmaking tool, amid warnings of artist displacement and environmental costs - [Kotaku](https://kotaku.com/george-miller-mad-max-ai-movies-film-babe-2000633726)

...

## AI Adoption by the Numbers

- ChatGPT hits 700M monthly, 800M weekly users and processes 2.5B messages daily, Altman says (July 2024) - [Business Insider](https://www.businessinsider.com/chatgpt-users-growth-openai-growth-sam-altman-ai-llm-2025-10)
- Salesforce, Shopify, Canva surpass 1T OpenAI tokens; AI music hits 12M users; buggy AI coding tools spark calls to rehire programmers - [Hacker Noon](https://hackernoon.com/10-9-2025-newsletter?source=rss)
- Users turn to ChatGPT for companionship and emotional support; communities say AI chats can aid mental health and social skills, deserving empathy over stigma - [Reddit](https://www.reddit.com/r/ChatGPT/comments/1o1pj6w/dont_shame_people_for_using_chatgpt_for/)

## Intel Bets Big on 18A

- Inside Intel's \\$20B+ Fab 52: ASML lithography powers comeback bid as customers await production proof against TSMC and Apple. - [The New York Times](https://www.nytimes.com/2025/10/09/technology/intels-big-bet-inside-the-chipmakers-make-or-break-factory.html)
- Intel’s 18A Panther Lake with RibbonFET, PowerVia promises 50% multi-thread uplift or 30% less power; broad availability January 2026. - [Tom's Hardware](https://www.tomshardware.com/pc-components/cpus/intel-takes-the-wraps-off-panther-lake-first-18a-client-processor-brings-the-best-of-lunar-lake-and-arrow-lake-together-in-one-package)
- Clearwater Forest: Intel’s 18A Xeon with up to 288 E-cores, 17% IPC uplift, built at Fab 52, launching H1 2026. - [Inside HPC](https://insidehpc.com/2025/10/intel-announces-18a-based-2nm-clearwater-forest-server-cpu-produced-at-fab-52-in-arizona/)

## From Chips to Grips

- Intel unveils Panther Lake chip and Robotics AI Suite to power energy-efficient humanoids and arms, delivering nearly 200 TOPS and retail order verification. - [Yahoo](https://finance.yahoo.com/news/intel-pushes-further-into-humanoid-robotics-with-new-chip-software-130023416.html)
- Figure unveils Figure 03 humanoid with new sensor and hand suite for Helix AI, home-safety features, and BotQ mass production up to 12,000 units annually. - [Figure](https://www.figure.ai/news/introducing-figure-03)
- Humanoid hands remain hard: Tesla, Figure, Unitree confront dexterity limits; Tesla trims fingers to 22 degrees of freedom and delays mass production. - [Fast Company](https://www.fastcompany.com/91418249/do-robots-dream-of-perfect-hands-optimus-musk-robot)

## Codex Commits More Code

- OpenAI launches GPT-5-Codex at DevDay 2025; GA boosts code contributions 70%, SDKs roll out, early adopters include Cisco - [VentureBeat](https://venturebeat.com/ai/the-most-important-openai-announcement-you-probably-missed-at-devday-2025)
- Study of 300k PRs: OpenAI Codex wins 74.3% approval vs Claude Code's 73.7%; adoption rising, usage gap shrinking - [The Information](https://www.theinformation.com/articles/openai-catching-anthropic-ai-coding)

## Voice Clones, Real Risks

- Study: With under 5 minutes of speech, listeners spot AI clones 58% and real voices 62%, signaling rising deepfake scam risk - [The Register](https://go.theregister.com/feed/www.theregister.com/2025/10/09/voice_clone_detection_study/)
- NeuTTS Air open-source on-device voice cloning enables real-time TTS on phones, laptops, Raspberry Pi, with neural audio codec and watermarking - [GitHub Gist](https://github.com/neuphonic/neutts-air)

In [31]:
state.newsletter_sections

{'Intel 18A Manufacturing': "## Intel Bets Big on 18A\n\n- Inside Intel's $20B+ Fab 52: ASML lithography powers comeback bid as customers await production proof against TSMC and Apple. - [The New York Times](https://www.nytimes.com/2025/10/09/technology/intels-big-bet-inside-the-chipmakers-make-or-break-factory.html)\n- Intel’s 18A Panther Lake with RibbonFET, PowerVia promises 50% multi-thread uplift or 30% less power; broad availability January 2026. - [Tom's Hardware](https://www.tomshardware.com/pc-components/cpus/intel-takes-the-wraps-off-panther-lake-first-18a-client-processor-brings-the-best-of-lunar-lake-and-arrow-lake-together-in-one-package)\n- Clearwater Forest: Intel’s 18A Xeon with up to 288 E-cores, 17% IPC uplift, built at Fab 52, launching H1 2026. - [Inside HPC](https://insidehpc.com/2025/10/intel-announces-18a-based-2nm-clearwater-forest-server-cpu-produced-at-fab-52-in-arizona/)",
 'AI Startup Funding': '## AI Funding Frenzy\n\n- xAI targets $20B SPV, with $2B Nvidia

In [ ]:
give section a standard string output
output sections
do the pruning 
rewrite other after pruning


In [14]:
input_text = headline_df.loc[headline_df["cat"]=="AI Business Value Gap"].sort_values("rating", ascending=False)[["rating", "short_summary", "site_name", "url"]].to_json(orient="records")
input_text


'[{"rating":9.5199365,"short_summary":"McKinsey finds only 30% of AI software vendors report quantifiable ROI from deployments; rising IT costs without matching labor cuts, weak change-management investment and complex pricing hinder adoption, prompting vendors to adopt hybrid per-user plus consumption-based pricing.","site_name":"The Register","url":"https:\\/\\/www.theregister.com\\/2025\\/10\\/09\\/mckinsey_ai_monetization\\/"},{"rating":9.4274351938,"short_summary":"The Bank of England warns global financial markets could face a sharp correction if investor sentiment sours on AI prospects or confidence in the U.S. Federal Reserve\'s independence, citing inflated share prices driven by AI enthusiasm.","site_name":"Biztoc","url":"https:\\/\\/biztoc.com\\/x\\/42d9125aea870810"},{"rating":9.0760579343,"short_summary":"BCG finds only 5% of over 1,250 global companies realize significant AI returns; software, telecommunications, and fintech lead, driven by long-term AI planning, leadersh

In [35]:
write_section_system_prompt, write_section_user_prompt, model = \
    LangfuseClient().get_prompt("newsagent/write_section")

write_section_agent = LLMagent(
    system_prompt=write_section_system_prompt,
    user_prompt=write_section_user_prompt,
    output_type=Section,
    model=model,
    verbose=True,
    logger=logger
)


INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/write_section' from Langfuse
INFO:llm:Parsed prompt 'newsagent/write_section': model=gpt-5, system_len=1860, user_len=22
16:07:45 | NewsletterAgent.newsletter_agent | INFO | Initialized LLMagent:
system_prompt: You are a newsletter editor transforming a collection of raw news stories into a compelling, coherent topic summary.

# TASK
Transform the list of news stories into a well-structured newsletter section with a strong title, crisp headlines, and punchy summaries.

# INPUT
- a list of json objects sorted by rating (highest first)
- Each item: { "rating": number, "short_summary": string, "site_name": string, "url": string }

# OUTPUT
- minified JSON only, in the specified schema, with section title and list of headlines, each with a list of links
- no code fences, no line breaks, no extra whitespace).

# WORKFLOW
1. **Section title**
- Infer the dominant, unifying topic of the set of stories (by co

In [36]:
response = await write_section_agent.run_prompt(input_text=input_text)
response

16:07:50 | NewsletterAgent.newsletter_agent | INFO | User message: STORIES:
[{"rating":9.5199365,"short_summary":"McKinsey finds only 30% of AI software vendors report quantifiable ROI from deployments; rising IT costs without matching labor cuts, weak change-management investment and complex pricing hinder adoption, prompting vendors to adopt hybrid per-user plus consumption-based pricing.","site_name":"The Register","url":"https:\/\/www.theregister.com\/2025\/10\/09\/mckinsey_ai_monetization\/"},{"rating":9.4274351938,"short_summary":"The Bank of England warns global financial markets could face a sharp correction if investor sentiment sours on AI prospects or confidence in the U.S. Federal Reserve's independence, citing inflated share prices driven by AI enthusiasm.","site_name":"Biztoc","url":"https:\/\/biztoc.com\/x\/42d9125aea870810"},{"rating":9.0760579343,"short_summary":"BCG finds only 5% of over 1,250 global companies realize significant AI returns; software, telecommunicatio

Section(section_title='AI Hype Meets Hard ROI', headlines=[Story(headline='McKinsey: Only 30% of AI vendors show measurable ROI; pricing shifts amid rising IT costs', links=[Link(url='https://www.theregister.com/2025/10/09/mckinsey_ai_monetization/', site_name='The Register')], prune=False), Story(headline='BCG: Just 5% of 1,250 companies achieve significant AI returns; software, telecom, fintech lead', links=[Link(url='https://www.businessinsider.com/industries-seeing-value-from-ai-bcg-consulting-report-2025-10', site_name='Business Insider')], prune=False), Story(headline='Bank of England warns AI-fueled market rally could sharply correct if sentiment turns or Fed credibility wobbles', links=[Link(url='https://biztoc.com/x/42d9125aea870810', site_name='Biztoc')], prune=False), Story(headline='China equities climb as AI buzz outshines weak consumer spending after market break', links=[Link(url='https://www.bloomberg.com/news/articles/2025-10-08/china-markets-reopen-to-show-if-ai-buzz-

In [39]:
display(Markdown(str(response)))

## AI Hype Meets Hard ROI

- McKinsey: Only 30% of AI vendors show measurable ROI; pricing shifts amid rising IT costs - [The Register](https://www.theregister.com/2025/10/09/mckinsey_ai_monetization/)
- BCG: Just 5% of 1,250 companies achieve significant AI returns; software, telecom, fintech lead - [Business Insider](https://www.businessinsider.com/industries-seeing-value-from-ai-bcg-consulting-report-2025-10)
- Bank of England warns AI-fueled market rally could sharply correct if sentiment turns or Fed credibility wobbles - [Biztoc](https://biztoc.com/x/42d9125aea870810)
- China equities climb as AI buzz outshines weak consumer spending after market break - [Bloomberg](https://www.bloomberg.com/news/articles/2025-10-08/china-markets-reopen-to-show-if-ai-buzz-outshines-soft-spending)
- CFOs report tangible AI ROI in finance - from forecasting to faster closes - while keeping humans in the loop - [PYMNTS](http://www.pymnts.com/artificial-intelligence-2/2025/cfos-embrace-ais-roi-as-finance-outgrows-cost-center-label/)
- Generative AI boosts coding like a junior teammate; engineers still create value through architecture, oversight, and security - [Hacker Noon](https://hackernoon.com/the-ai-writes-code-but-the-engineer-creates-value?source=rss)
- 46% of European SMEs use AI daily, yet many lack basic digital tools, study warns - [Live Mint](https://www.livemint.com/companies/news/european-small-businesses-rush-into-ai-without-basic-digital-tools-study-shows-11759916332271.html)
- Tech brands slap AI on basic gadgets, risking hype dilution and needless complexity, experts warn - [CNET](https://www.cnet.com/home/smart-home/so-many-brands-are-calling-their-basic-tech-ai-now-heres-why-im-worried/)
- Fourteen years on, Apple races to rebuild Siri, testing external models to finally deliver a smarter assistant - [Macworld](https://www.macworld.com/article/2935321/siri-anniversary-apple-intelligence-future-devices.html)
- Tesla scales back 2025 Optimus robot ambitions amid major technical hurdles, especially the hands - [The Information](https://www.theinformation.com/articles/elon-musk-preps-teslas-optimus-prime-time-big-hurdles-remain)
- AI vibe marketing promises emotion-coded campaigns for small businesses, but challenges traditional agencies - [Forbes](https://www.forbes.com/sites/michaelashley/2025/10/08/the-ai-marketing-solution-that-thinks-in-feelings-not-keywords/)
- YC startup seeks founding ML engineer to build multimodal forecasting foundation models - [Hacker News](https://www.ycombinator.com/companies/the-forecasting-company/jobs/cXJzAhA-founding-machine-learning-engineer)

In [34]:
class Link(BaseModel):
    url: str = Field(description="URL of the article")
    site_name: str = Field(description="Name of the website/source")
    def __str__(self):
        return f"[{self.site_name}]({self.url})"


class Story(BaseModel):
    headline: str = Field(description="Summary of the story")
    links: List[Link] = Field(description="List of links related to this story")
    prune: bool = Field(description="Whether to prune/exclude this story")
    def __str__(self):
        return "" if self.prune else f"- {self.headline} - " + " ".join([str(s) for s in self.links])


class Section(BaseModel):
    section_title: str = Field(description="Title of the newsletter section")
    headlines: List[Story] = Field(description="List of stories in this section")
    def __str__(self):
        return f"## {self.section_title}\n\n" + "\n".join(
            [str(h) for h in self.headlines]
        )
        

In [22]:
headline_df=state.headline_df
headline_df.loc[headline_df["cat"]=="AI Business Value Gap"].sort_values("rating", ascending=False)[["rating", "short_summary", "site_name", "url"]].to_json(orient="records")


'[{"rating":9.5199365,"short_summary":"McKinsey finds only 30% of AI software vendors report quantifiable ROI from deployments; rising IT costs without matching labor cuts, weak change-management investment and complex pricing hinder adoption, prompting vendors to adopt hybrid per-user plus consumption-based pricing.","site_name":"The Register","url":"https:\\/\\/www.theregister.com\\/2025\\/10\\/09\\/mckinsey_ai_monetization\\/"},{"rating":9.4274351938,"short_summary":"The Bank of England warns global financial markets could face a sharp correction if investor sentiment sours on AI prospects or confidence in the U.S. Federal Reserve\'s independence, citing inflated share prices driven by AI enthusiasm.","site_name":"Biztoc","url":"https:\\/\\/biztoc.com\\/x\\/42d9125aea870810"},{"rating":9.0760579343,"short_summary":"BCG finds only 5% of over 1,250 global companies realize significant AI returns; software, telecommunications, and fintech lead, driven by long-term AI planning, leadersh

In [41]:
print("# SUGGESTED TOPICS:")
catcount = headline_df.groupby("cat").count().reset_index()[['cat', 'source']].sort_values('source', ascending=False)
for c in catcount["cat"]:
    print(c)
print()
print("# RAW NEWS ITEMS:")
i =0
for row in headline_df.sort_values(["cat", "rating"], ascending=False).itertuples():
    print(f"[{row.title}]({row.url}) - {row.site_name}\n".replace("$","\\\\$"))
    row_topics = ", ".join(row.topics)
    print(f"Topics: {row_topics}\n".replace("$","\\\\$"))
    print(f"Rating: {row.rating:.1f}\n")    
    print(f"{row.short_summary}\n".replace("$","\\\\$"))
    print(f"{row.summary}\n".replace("$","\\\\$"))
    print("~~~\n")
    i +=1


# SUGGESTED TOPICS:
AI Societal Impact
AI Adoption And Funding Surge
Other
AI Business Value Gap
Instant AI Video Generators
AI Legal Services
OpenAI Massive Compute Deals
OpenAI AgentKit Launch
TSMC AI Revenue Boom
US AI Policy Patchwork
EU AI Sovereignty Strategy
Agent Memory Frameworks
ChatGPT Apps Platform Play
AI Music Industry Disruption
AI Lawsuit Settlement Funding
Threat Actors Use AI Tools
ChatGPT Asia Expansion

# RAW NEWS ITEMS:
[U.S. Approves Nvidia AI Chip Exports to UAE Under Bilateral Tech Pact](https://hackernoon.com/us-approves-nvidia-ai-chip-exports-to-uae-under-bilateral-tech-pact?source=rss) - Hacker Noon

Topics: Nvidia, AI Chip Exports, US Middle East Relations, Policy And Regulation, Governance, Deals, Data Centers

Rating: 10.1

The U.S. authorized Nvidia to export advanced AI chips worth billions to the UAE under a bilateral AI agreement signed earlier this year, enabling reciprocal UAE investments in the U.S. and support for large-scale data centers to advanc

In [42]:
headline_df['cat'].unique() 


array(['AI Legal Services', 'Instant AI Video Generators',
       'AI Adoption And Funding Surge', 'AI Lawsuit Settlement Funding',
       'AI Business Value Gap', 'Other', 'OpenAI Massive Compute Deals',
       'EU AI Sovereignty Strategy', 'AI Music Industry Disruption',
       'Agent Memory Frameworks', 'ChatGPT Asia Expansion',
       'US AI Policy Patchwork', 'ChatGPT Apps Platform Play',
       'AI Societal Impact', 'TSMC AI Revenue Boom',
       'OpenAI AgentKit Launch', 'Threat Actors Use AI Tools'],
      dtype=object)

In [ ]:
astate.get_completed_steps() 



In [ ]:
# User prompt to run workflow
user_prompt = "Show the workflow status"

print(f"\n📝 User prompt: '{user_prompt}'")
print("=" * 80)

# Run the agent with persistent state
start_time = time.time()
result = await agent.run_step(user_prompt)
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)

In [29]:
headline_df=state.headline_df
i =0
for row in headline_df.sort_values("rating", ascending=False).itertuples():
    display(Markdown(f"{row.rating:.1f}"))   
    display(Markdown(f"[{row.title}]({row.url}) - {row.site_name}".replace("$","\\\\$")))
    row_topics = ", ".join(row.topics)
    display(Markdown(f"Topics: {row_topics}".replace("$","\\\\$")))
    display(Markdown(f"{row.short_summary}".replace("$","\\\\$")))
    display(Markdown(f"{row.summary}".replace("$","\\\\$")))
    i +=1
#     if i>=30:
#         break

16.3

[Bank of England warns AI stock bubble rivals 2000 dotcom peak](https://arstechnica.com/ai/2025/10/bank-of-england-warns-ai-stock-bubble-rivals-2000-dotcom-peak/) - Ars Technica

Topics: Tech Stocks, Market Concentration, Finance, Policy and Regulation, Bubble, UK, Gen AI

Bank of England warns AI-driven valuations resemble the 2000 dotcom peak, noting Nvidia, Microsoft, Apple, Amazon and Meta account for 30% of the S&P 500—the highest market concentration in 50 years—and could spill over into Britain’s financial system.

- The Bank of England warns that the current AI-driven stock market valuations resemble the peak of the 2000 dotcom bubble, signaling increased risk of a sharp market correction.
- AI-focused companies constitute an unprecedented portion of market value, with five major firms—Nvidia, Microsoft, Apple, Amazon, and Meta—accounting for 30% of the S&P 500's valuation, marking the highest concentration in 50 years.
- The Bank of England highlighted that a negative shift in investor sentiment toward AI could have material spillover effects on Britain’s financial system, emphasizing concerns about market stability amid soaring AI investments.

...

0.6

[When Curl Works but IntelliJ Doesn't: The Ollama Connection Mystery](https://blog.tymscar.com/posts/intellijollamaconnectionmystery/) - 

Topics: IntelliJ, JVM Network, Programming Debugging, Networking, Java Network, Gen AI, Coding Assistants

IntelliJ failed to connect to a locally reachable Ollama instance because the JVM attempted IPv6 on an IPv4-only LAN, producing No route to host; adding -Djava.net.preferIPv4Stack=true forced IPv4 and restored connectivity without other changes.

- IntelliJ IDEA failed to connect to the Ollama instance despite curl successfully accessing it over both HTTPS and HTTP, due to JVM network stack behavior.
- The root cause was the JVM attempting an IPv6 connection first on an IPv4-only LAN, resulting in a "No route to host" error; this was resolved by adding the JVM option -Djava.net.preferIPv4Stack=true.
- The fix disabled IPv6 usage in the JVM, allowing IntelliJ to connect instantly to Ollama without changes to Traefik, TLS certificates, or network configuration, highlighting the importance of checking JVM network preferences in similar Java application connectivity issues.

In [ ]:
class DistilledStory(BaseModel):
    """DistilledStory class for structured output distillation into a single sentence """
    item: str = Field(description="List of StoryRating")
        
system, user, model = LangfuseClient().get_prompt("newsagent/item_distiller")

distill_agent = LLMagent(
            system_prompt=system,
            user_prompt=user,
            output_type=DistilledStory,
            model=model,
            verbose=False,
            logger=logger
        )

In [ ]:
response = await distill_agent.run_prompt(input_text="""AI 'Homeless Man' Challenge Sparks Outrage as Police Called Over Dangerous Viral Trend - International Business Times

Topics: AI Pranks, Public Safety, Ethical Concerns, Disinformation, Policy And Regulation, Snapchat Challenges, Gen AI

Rating: 1.5

Topics: AI Pranks, Public Safety, Ethical Concerns, Disinformation, Policy And Regulation, Snapchat Challenges, Gen AI

Parents in shock after dangerous 'homeless man' Snapchat AI prank goes horribly wrong, police called to calm the viral chaos.

A viral AI-driven prank called the 'homeless man' challenge on Snapchat deceived parents into thinking a homeless person had broken into their home, prompting police intervention.
The prank caused significant public backlash due to the misuse of AI technology to create real emergency scares, leading to police questioning the pranksters and debates about the legal consequences.
Experts emphasize the ethical concerns and potential emotional harm caused by such digital pranks, highlighting the need for responsible use of AI to avoid wasting emergency resources and creating community panic.
""")
response

In [19]:
headline_df=state.headline_df
i =0
for row in headline_df.sort_values("rating", ascending=False).itertuples():
    print(f"[{row.title}]({row.url}) - {row.site_name}\n".replace("$","\\\\$"))
    row_topics = ", ".join(row.topics)
    print(f"Topics: {row_topics}\n".replace("$","\\\\$"))
    print(f"{row.short_summary}\n".replace("$","\\\\$"))
    print(f"{row.summary}\n".replace("$","\\\\$"))
    print("~~~\n")

    i +=1

[SoftBank to buy ABB’s robotics unit in \\$5.4bn deal](https://www.ft.com/content/4080ecae-0654-4597-9596-025c373874c3) - Financial Times

Topics: SoftBank, Robotics, Mergers and Acquisitions, Artificial Superintelligence, Hardware, Manufacturing, Japan

SoftBank will acquire ABB's robotics unit for \\$5.4 billion, aligning with founder Masayoshi Son's vision to advance artificial superintelligence and expand SoftBank's robotics and AI capabilities.

- SoftBank is set to acquire ABB's robotics unit for \\$5.4 billion as part of its strategic expansion into robotics technology.
- The acquisition supports SoftBank founder Masayoshi Son's vision of achieving 'artificial superintelligence,' reflecting significant business growth and innovation ambitions.
- This deal positions SoftBank as a major player in the robotics market, with expected implications for advancing AI-integrated robotic solutions across industries.

~~~
[Google’s latest AI model uses a web browser like you do](https://www

TODO:
- update final prompt
- output sections using short summary
# SUGGESTED TOPICS
AI Agents And Reliability
AI Creative Industry Impact
AI Development Tools And Standards
AI Market Valuations
AI Phishing Surge
AI Security Risks
AI Workforce Impact
C2PA Image Provenance
Circular Deal Inflation
Cross-Industry AI Adoption
Crunch Lab Decentralized AI
Data Center Environmental Impact
Deepfake Video Ethics
Dell Raises AI Forecasts
EU AI Strategy
Enterprise AI Data Leakage
Enterprise AI Partnerships
Google Gemini 2.5
Healthcare AI Investments
OpenAI Platform Issues
Other
Qualcomm Acquires Arduino
SoftBank Acquires ABB Robotics
Youth Support AI Ethics

- take each summary and boil it down to 1 sentence , output correct format
- initial write sections - prompt and output json for each section asynchronously
- check and rewrite each section for format asynchronsously
- assemble sections
- do a critic loop